<font face="微软雅黑" size=4 color=#000099 > 1. 导入需要用到的库


In [11]:
import pandas as pd
import numpy as np
import re

file_path= "data/test.xlsx"

<font face="微软雅黑" size=4 color=#000099 > 2. 运行自定义的函数

<font face="微软雅黑" size=3 color=black >2.1 填写资产负债表前半部分特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=black>a. 流动资产中：
    <br>
    <font face="微软雅黑" size=2 color=blue>{结算备付金，拆出资金，持有待售资产} => 其他流动资产
    <br>
    <font face="微软雅黑" size=2 color=blue>{以公允价值计量且其变动计入当期损益的金融资产，衍生金融资产} => 交易性金融资产
    <br>
    <font face="微软雅黑" size=2 color=blue>{应收保费，应收分保账款，应收分保合同准备金,买入返售金融资产} => 其他应收款
    <br>
    <font face="微软雅黑" size=2 color=blue>{应收款项融资}=> 应收账款
    <br>
    <font face="微软雅黑" size=2 color=black>b. 非流动资产中：
    <br>
    <font face="微软雅黑" size=2 color=blue>{发放贷款和垫款，债权投资，其他债权投资} => 持有至到期投资
    <br>
    <font face="微软雅黑" size=2 color=blue>{其他权益工具投资，其他非流动金融资产} =>长期股权投资
    <br>
    <font face="微软雅黑" size=2 color=blue>{使用权资产} => 无形资产

In [58]:
#针对填写资产负债表清洗读取结果
def clean_up_bal(raw_sheet):
    
    for index, row in raw_sheet.iterrows():
        row['项目'] = re.sub('△',"",row['项目'])
        row['项目'] = re.sub(' ',"",row['项目'])
        row['项目'] = re.sub('☆',"",row['项目'])
        row['项目'] = re.sub('其中：',"",row['项目'])
        row['项目'] = re.sub('\*',"",row['项目'])
        row['项目'] = re.sub('减：',"",row['项目'])
        row['项目'] = re.sub('加：',"",row['项目'])       
        
        if row['项目'] == "实收资本" or row['项目'] == "实收资本（或股本）净额" or row['项目'] == "实收资本净额":
            row['项目'] = "实收资本（或股本）"
        elif row ['项目'] == "所有者权益合计":
            row['项目'] = "所有者权益（或股东权益）合计"
        elif row ['项目'] == "负债和所有者权益总计":
            row['项目'] = "负债和所有者权益（或股东权益）总计"

#填写资产负债表前半部分
def fill_bal_1(balance_sheet, clean_sheet):
    
    other_current_assets = ['结算备付金','拆出资金','持有待售资产']
    trading_financial_assets=['以公允价值计量且其变动计入当期损益的金融资产','衍生金融资产']
    other_receivables = ['应收保费','应收分保账款','应收分保合同准备金']
    accounts_receivables = ['应收款项融资']
    held_to_maturity_investments = ['发放贷款和垫款','债权投资','其他债权投资']
    long_term_investment_on_stocks = ['其他权益工具投资','其他非流动金融资产']
    intangible_asstes= ['使用权资产']
    
    special= ["其他流动资产","交易性金融资产","其他应收款","应收账款","持有至到期投资","长期股权投资","无形资产"]
    
    oca=[]
    tfa=[]
    orc=[]
    arc=[]
    htmi=[]
    ltis=[]
    ita=[]
    
    #遍历排列好的行
    for index, row in clean_sheet.iterrows():
        num = row['期末余额']
        if row['项目'] in other_current_assets:
            oca.append(num)
        elif row['项目'] in trading_financial_assets:
            tfa.append(num)
        elif row['项目'] in other_receivables:
            orc.append(num)
        elif row['项目'] in accounts_receivables:
            arc.append(num)
        elif row['项目'] in held_to_maturity_investments :
            htmi.append(num)
        elif row['项目'] in long_term_investment_on_stocks:
            ltis.append(num)
        elif row['项目'] in intangible_asstes:
            ita.append(num)
        elif row['项目'] == "其他流动资产":
            num+=sum(oca)
        elif row['项目'] == "交易性金融资产":
            num+=sum(tfa)
        elif row['项目'] == "其他应收款":
            num+=sum(orc)
        elif row['项目'] == "应收账款":
            num+=sum(arc)
        elif row['项目'] == "持有至到期投资":
            num+=sum(htmi)
        elif row['项目'] == "长期股权投资":
            num+=sum(ltis)    
        elif row['项目'] == "无形资产":
            num+=sum(ita)
                             
        balance_sheet.loc[balance_sheet[1]==row['项目'] ,[2]]=num

<font face="微软雅黑" size=3 color=black >2.2 填写资产负债表后半部分特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=black>a. 流动负债中：
    <br>
    <font face="微软雅黑" size=2 color=blue>['向中央银行借款','拆入资金'] => '短期借款’
    <br>
    <font face="微软雅黑" size=2 color=blue>['以公允价值计量且其变动计入当期损益的金融负债','衍生金融负债','卖出回购金融资产款','吸收存款及同业存放','代理买卖证券款','代理承销证券款'] => '交易性金融负债’
    <br>
    <font face="微软雅黑" size=2 color=blue>['应付手续费及佣金','应付分保账款'] => '其他应付款’
    <br>
    <font face="微软雅黑" size=2 color=blue>['持有待售负债']=> '其他流动负债’
    <br>
    <font face="微软雅黑" size=2 color=black>b. 非流动负债中：
    <br>
    <font face="微软雅黑" size=2 color=blue>['租赁负债','长期应福职工薪酬'] => ‘长期应付款’
    <br>
    <font face="微软雅黑" size=2 color=blue>['保险合同准备金','递延收益']=> ‘其他非流动负债’<br>
    <font face="微软雅黑" size=2 color=black>c. 所有者权益中：
    <br>
    <font face="微软雅黑" size=2 color=blue>['其他权益工具','其他综合收益','专项储备','少数股东权益'] => '资本公积’
    <br>
    <font face="微软雅黑" size=2 color=blue>['一般风险准备'] => '盈余公积’

In [65]:
#填写资产负债表后半部分
def fill_bal_2(balance_sheet,clean_sheet):
    
    short_term_liabilities =['向中央银行借款','拆入资金']
    trading_financial_liabilities = ['以公允价值计量且其变动计入当期损益的金融负债','衍生金融负债','卖出回购金融资产款','吸收存款及同业存放','代理买卖证券款','代理承销证券款']
    other_payables = ['应付手续费及佣金','应付分保账款']
    other_current_liabilities = ['持有待售负债']
    
    long_term_payables = ['租赁负债','长期应福职工薪酬']
    other_non_current_liabilities=['保险合同准备金','递延收益']
    
    capital_reserves = ['其他权益工具','其他综合收益','专项储备','少数股东权益']
    earned_surplus = ['一般风险准备']
    
    special= ["短期借款","交易性金融负债","其他应付款","其他流动负债","长期应付款","其他非流动负债","资本公积","盈余公积"]
    
    stl=[]
    tfl=[]
    op=[]
    ocl=[]
    
    ltp=[]
    oncl=[]
    
    cr=[]
    es=[]
    
    
    #遍历排列好的行
    for index, row in clean_sheet.iterrows():
        num = row['期末余额']
                    
        if row['项目'] in short_term_liabilities:
            stl.append(num)
        elif row['项目'] in trading_financial_liabilities:
            tfl.append(num)
        elif row['项目'] in other_payables:
            op.append(num)
        elif row['项目'] in other_current_liabilities:
            ocl.append(num)
        elif row['项目'] in long_term_payables:
            ltp.append(num)
        elif row['项目'] in other_non_current_liabilities:
            oncl.append(num)
        elif row['项目'] in  capital_reserves:
            cr.append(num)
        elif row['项目'] in  earned_surplus:
            es.append(num)

        elif row['项目'] == "短期借款":
            num+=sum(stl)
        elif row['项目'] == "交易性金融负债":
            num+=sum(tfl)
        elif row['项目'] == "其他应付款":
            num+=sum(op)
        elif row['项目'] == "其他流动负债":
            num+=sum(ocl)
        elif row['项目'] == "长期应付款":
            num+=sum(ltp)
        elif row['项目'] == "其他非流动负债":
            num+=sum(oncl)
        elif row['项目'] == "资本公积":
            num+=sum(cr)
        elif row['项目'] == "盈余公积":
            num+=sum(es)

        balance_sheet.loc[balance_sheet[1]==row['项目'] ,[2]]=num

<font face="微软雅黑" size=3 color=black >2.3 填写利润表特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=blue>['利息支出','手续费及佣金支出','持续经营净利润','退保金','赔付支出净额','提取保险责任准备金净额','保单红利支出','分保费用','研发费用'] => '管理费用'
    <br>
    <font face="微软雅黑" size=2 color=blue>['汇兑收益','净敞口套期收益','公允价值变动收益','信用减值损失','资产处置收益'] => '其他收益'

In [84]:
#针对填写利润表清洗读取结果
def clean_up_income(raw_sheet):
    
    for index, row in raw_sheet.iterrows():
        row['项目'] = re.sub('△',"",row['项目'])
        row['项目'] = re.sub(' ',"",row['项目'])
        row['项目'] = re.sub('☆',"",row['项目'])
        if row['项目'] != "其中：营业收入" and  row['项目'] != "其中：对联营企业和合营企业的投资收益":
            row['项目'] = re.sub('其中：',"",row['项目'])
        row['项目'] = re.sub('\*',"",row['项目'])
        row['项目'] = re.sub('减：',"",row['项目'])
        row['项目'] = re.sub('加：',"",row['项目'])  
        row['项目'] = re.sub('（损失以“-”号填列）',"",row['项目']) 
        row['项目'] = re.sub('（净收益以“-”号填列）',"",row['项目']) 
                
        if row['项目'] == "税金及附加":
            row['项目'] = "营业税金及附加"
        elif row ['项目'] == "一、营业总收入":
            row['项目'] = '营业收入'
        elif "营业利润" in  row['项目']:
            row['项目']='营业利润'
        elif "利润总额" in  row['项目']:
            row['项目']='利润总额'
        elif row['项目']=='五、净利润（净亏损以“－”号填列）' :
            row['项目']='净利润'    
        elif row['项目'] =='八、每股收益：':
            row['项目']='每股收益：' 
        elif row['项目'] =='资产减值损失':
            row ['本期金额']=-row ['本期金额']
            
    
#填写利润表
def fill_income(income_statement,clean_sheet):
    
    admin_expense = ['利息支出','手续费及佣金支出','持续经营净利润','退保金','赔付支出净额','提取保险责任准备金净额','保单红利支出','分保费用','研发费用']
    other_benefits=['汇兑收益','净敞口套期收益','信用减值损失','资产处置收益']
    
    special= ['管理费用', '其他收益']

    ae=[]
    ob=[]
    

    #遍历排列好的行
    for index, row in clean_sheet.iterrows():
        num = row['本期金额']

        if row['项目'] in admin_expense:
            ae.append(num)
        elif row['项目'] in other_benefits:
            ob.append(num)
        elif row['项目'] == "管理费用":
            num+=sum(ae)
        elif row['项目'] == "其他收益":
            num+=sum(ob)

        income_statement.loc[income_statement[1]==row['项目'] ,[2]]=num

<font face="微软雅黑" size=3 color=black >2.4 填写现金流量表特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=blue>['客户存款和同业存放款项净增加额','向中央银行借款净增加额','向其他金融机构拆入资金净增加额','收到原保险合同保费取得的现金','收到再保业务现金净额','保户储金及投资款净增加额','处置以公允价值计量且其变动计入当期损益的金融资产净增加额','收取利息'、'手续费及佣金的现金','拆入资金净增加额','回购业务资金净增加额','代理买卖证券收到的现金净额'] => '收到其他与经营活动有关的现金'
    <br>
    <font face="微软雅黑" size=2 color=blue>['客户贷款及垫款净增加额','存放中央银行和同业款项净增加额','支付原保险合同赔付款项的现金','拆出资金净增加额','支付利息、手续费及佣金的现金','支付保单红利的现金'] => '支付其他与经营活动有关的现金'
    <br>
    <font face="微软雅黑" size=2 color=blue>['质押贷款净增加额']=>'支付其他与投资活动有关的现金’
     

In [98]:
#针对填写现金流量表清洗读取结果
def clean_up_cash(raw_sheet):
    
    for index, row in raw_sheet.iterrows():
        row['项目'] = re.sub('△',"",row['项目'])
        row['项目'] = re.sub(' ',"",row['项目'])
        row['项目'] = re.sub('☆',"",row['项目'])
        row['项目'] = re.sub('其中：',"",row['项目'])
        row['项目'] = re.sub('\*',"",row['项目'])
        row['项目'] = re.sub('所',"",row['项目'])
        row['项目'] = re.sub('加：',"",row['项目'])

        if row['项目']== "四、汇率变动对现金及现金等价物的影响" :
            row['项目']='汇率变动对现金及现金等价物的影响'
        elif row['项目']== "五、现金及现金等价物净增加额" :
            row['项目']='现金及现金等价物净增加额'
        elif row['项目']== "六、期末现金及现金等价物余额" :
            row['项目']='期末现金及现金等价物余额'

#填写现金流量表
def fill_cash_flow(cash_flow_statement,clean_sheet):
    
    cash_inflows_operating = ['客户存款和同业存放款项净增加额','向中央银行借款净增加额','向其他金融机构拆入资金净增加额','收到原保险合同保费取得的现金','收到再保业务现金净额','保户储金及投资款净增加额','处置以公允价值计量且其变动计入当期损益的金融资产净增加额','收取利息、手续费及佣金的现金','拆入资金净增加额','回购业务资金净增加额','代理买卖证券收到的现金净额']
    cash_outflows_operating = ['客户贷款及垫款净增加额','存放中央银行和同业款项净增加额','支付原保险合同赔付款项的现金','拆出资金净增加额','支付利息、手续费及佣金的现金','支付保单红利的现金']
    cash_outflows_investing = ['质押贷款净增加额']
    
    special= [ '收到其他与经营活动有关的现金' ,'支付其他与经营活动有关的现金' ,'支付其他与投资活动有关的现金']

    cio=[]
    coo=[]
    coi=[]
    

    #遍历排列好的行
    for index, row in clean_sheet.iterrows():
        num = row['本期金额']

        if row['项目']  in cash_inflows_operating:
            cio.append(num)
        elif row['项目']  in cash_outflows_operating:
            coo.append(num)
        elif row['项目']  in cash_outflows_investing:
            coi.append(num)
        elif row['项目']  == "收到其他与经营活动有关的现金":
            num+=sum(cio)
        elif row['项目']  == "支付其他与经营活动有关的现金":
            num+=sum(coo)
        elif row['项目']  == "支付其他与投资活动有关的现金":
            num+=sum(coi)

        cash_flow_statement.loc[cash_flow_statement[1]==row['项目']  ,[2]]=num

<font face="微软雅黑" size=4 color=#000099 > 3. 读取资产负债表数据并写入
    <br><br>
    <font face="微软雅黑" size=3 color=black > a.资产负债表第一部分

In [59]:
#读取资产负债表
balance_sheet = pd.read_excel('statement/客户财务报表－企业类.xls', '资产负债表')

bal_1 = pd.read_excel(file_path,'1')
bal_1 =bal_1 .dropna()

clean_up_bal(bal_1 )

fill_bal_1(balance_sheet, bal_1)
balance_sheet

,1,2
0,货币资金,112759638.26
1,交易性金融资产,0
2,应收票据,0
3,应收账款,0
4,预付款项,15764277.92
...,...,...
57,库存股,NaN
58,盈余公积,NaN
59,未分配利润,NaN
60,所有者权益（或股东权益）合计,NaN


<font face="微软雅黑" size=3 color=black > b.资产负债表第二部分

In [66]:
#读取资产负债表第一部分图片信息
bal_2 = pd.read_excel(file_path,'2')
bal_2 =bal_2 .dropna()

clean_up_bal(bal_2 )

fill_bal_2(balance_sheet, bal_2)

balance_sheet=balance_sheet.fillna(0)
balance_sheet

,1,2
0,货币资金,112759638.26
1,交易性金融资产,0
2,应收票据,0
3,应收账款,0
4,预付款项,15764277.92
...,...,...
57,库存股,0
58,盈余公积,0
59,未分配利润,234464.42
60,所有者权益（或股东权益）合计,1138234464.42


<font face="微软雅黑" size=4 color=#000099 > 4. 读取利润表数据并写入

In [86]:
income_sheet = pd.read_excel('statement/客户财务报表－企业类.xls', '利润表')
income = pd.read_excel(file_path,'3')
income = income.dropna()

clean_up_income(income)

fill_income(income_sheet, income)

income_sheet=income_sheet.fillna(0)
income_sheet

,1,2
0,营业收入,92239895.14
1,营业成本,71332036.2
2,营业税金及附加,0.0
3,销售费用,0.0
4,管理费用,0.0
5,财务费用,22637605.63
6,资产减值损失,0.0
7,公允价值变动收益,0.0
8,其他收益,2042365.92
9,投资收益,0.0


<font face="微软雅黑" size=4 color=#000099 >5. 读取现金流量表数据并写入

In [99]:
cash_sheet = pd.read_excel('statement/客户财务报表－企业类.xls', '现金流量表')
cash = pd.read_excel(file_path,'4')
cash  = cash .dropna()

clean_up_cash(cash )

fill_cash_flow(cash_sheet,cash)

cash_sheet=cash_sheet.fillna(0)
cash_sheet

,1,2
0,销售商品、提供劳务收到的现金,1.285838e+08
1,收到的税费返还,0.000000e+00
2,收到其他与经营活动有关的现金,0.000000e+00
3,经营活动现金流入小计,1.285838e+08
4,购买商品、接受劳务支付的现金,0.000000e+00
5,支付给职工以及为职工支付的现金,0.000000e+00
6,支付的各项税费,2.591917e+05
7,支付其他与经营活动有关的现金,6.776283e+06
8,经营活动现金流出小计,2.899468e+07
9,经营活动产生的现金流量净额,9.958910e+07


<font face="微软雅黑" size=4 color=#000099 >6. 填入Excel

In [100]:
excel_writer = pd.ExcelWriter('企业类处理结果.xls')

balance_sheet.to_excel(excel_writer,header=None,index=None,sheet_name="资产负债表")
income_sheet.to_excel(excel_writer,header=None,index=None,sheet_name="利润表")
cash_sheet.to_excel(excel_writer,header=None,index=None,sheet_name="现金流量表")

excel_writer.save()
excel_writer.close()

C:\Users\10491\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  if (await self.run_code(code, result,  async_=asy)):
